In [58]:
import geopandas as gpd
import numpy as np
import pandas as pd
from helper_functions.tower_helper import read_tower_data
from helper_functions.map_helper import read_map_district
from helper_functions.tower_clustering import cluster_towers, get_centroids
from helper_functions.voronoi_helper import from_towers_to_voronoi
import warnings
district_maps = "/Users/bilgecagaydogdu/Desktop/Geodirectory/turkey_administrativelevels0_1_2/tur_polbnda_adm2.shp"
tower_location="/Volumes/Extreme Pro/Cell_Tower_Locations/cell_city_district.txt"
voronoi_location="/Volumes/Extreme Pro/Cell_Tower_Locations/turkcell_voronoi/voronoi.shp"
cluster_location='/Volumes/Extreme Pro/Cell_Tower_Locations/clustered_towers/clusters.shp'

In [49]:
df_tower=read_tower_data(tower_location,voronoi_location)
gdf_districts = read_map_district(district_maps)
radians= cluster_towers(df_tower, dist_metric='haversine', t=1)
df_centroids=get_centroids(radians)
gdf_centroids=gpd.GeoDataFrame(df_centroids, geometry=gpd.points_from_xy(df_centroids.centroid_LNG,
                                                                                  df_centroids.centroid_LAT),
                                 crs="EPSG:4326")
gdf_centroids=gdf_centroids[['site_id','cluster','centroid_LNG','centroid_LAT']]
gdf_centroids=gdf_centroids.rename(columns={'centroid_LNG':'cluster_lng','centroid_LAT':'cluster_lat'})
gdf_centroids=gpd.GeoDataFrame(gdf_centroids, geometry=gpd.points_from_xy(gdf_centroids.cluster_lng,
                                                                                  gdf_centroids.cluster_lat),
                                 crs="EPSG:4326")
gdf_centroids=gdf_centroids.drop_duplicates(subset='cluster').reset_index(drop=True)
gdf_centroids.to_file('/Volumes/Extreme Pro/Cell_Tower_Locations/clustered_towers/clusters.shp')
gdf_centroids[['site_id','cluster']].to_csv('/Volumes/Extreme Pro/Cell_Tower_Locations/site_cluster_match.csv')

In [75]:
gdf_centroids=gpd.read_file(cluster_location)

In [55]:
df_tower=read_tower_data(tower_location,voronoi_location)

In [59]:
gdf, poly_shapes_list =from_towers_to_voronoi(cluster_location, district_maps,crs="EPSG:4326")

/Users/bilgecagaydogdu/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/bilgecagaydogdu/Desktop/EARTHQUAKE/mobile_phone_indicators/helper_functions/voronoi_helper.py:60: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  boundary_shape = cascaded_union(boundary.geometry)


[[28.98790111 41.03787859]
 [28.95109209 41.05168977]
 [28.97499646 41.04178012]
 ...
 [32.22945162 41.35474121]
 [34.70373969 40.73142496]
 [30.06022685 37.37213161]]


In [60]:
gdf=gdf.set_geometry('voronoi_geometry')

In [76]:
gdf_merged=gpd.GeoDataFrame(gdf_centroids.merge(gdf[['cluster','voronoi_geometry']],on='cluster',how='left'),geometry='geometry')

In [81]:
gdf_merged=gdf_merged.set_geometry('voronoi_geometry')

In [84]:
gdf_merged.to_csv('/Volumes/Extreme Pro/Cell_Tower_Locations/clustered_towers/cluster_voronoi.csv')